# This is a Movie Recommender System ( For practice Use Only ) 

In [ ]:
import pandas as pd
import numpy as np

__Loading in the data frames__

In [ ]:
user_ratings_df = pd.read_csv("user.csv")


user_ids = user_ratings_df["Unnamed: 0"]

movie_correls_df = user_ratings_df.drop("Unnamed: 0",axis = 1).corr()


In [ ]:
movie_ratings_df = pd.read_csv("movie.csv")
movie_ids = movie_ratings_df["Unnamed: 0"]

new_movie_df = movie_ratings_df.drop(101,axis = 0)
new_movie_df.drop(100,axis = 0,inplace = True)
new_movie_df

user_correls_df = new_movie_df.drop("Unnamed: 0",axis = 1).corr()

*Every Method I Used*

In [ ]:
def correl(user1,user2):
    return user_correls_df[user1][user2]

def usersContribution(main,other):
    return movie_ratings_df[other][0:101]*correl(main,other)

def getTopN(column,k):
    temp_df = user_correls_df.nlargest(k,columns = column)
    return temp_df[1:k]

def correlsOfContributingUsers(main,k):
    indexes = getTopN(main,k).index
    
    correls = []
    
    for user in indexes:
            temp = []
            contributions = usersContribution(main,user).fillna(0)
            for rating in contributions:
                if rating == 0:
                    temp.append(0)
                else:
                    temp.append(correl(user,main))
                    
            correls.append(temp)
            
    correls = pd.DataFrame(np.asarray(correls))
    return correls.sum(axis = 0)[0:100]
        

def sumOfContributions(main,k):
    indexes = getTopN(main,k).index
    
    weightOfUsers = []
    
    for user in indexes:
        weightOfUsers.append(usersContribution(main,user))
        
    weightOfUsers = pd.DataFrame(np.asarray(weightOfUsers),index = indexes,columns = movie_ids[0:101]).fillna(0)
    return weightOfUsers.sum(axis = 0)[0:100]

*GET N NEIGHBORS OF A USER*

In [373]:
up = sumOfContributions("3867",6)
down = correlsOfContributingUsers("3867",6)

predictions = pd.DataFrame(up.values/down.values,index = movie_ids[0:100],columns = ["Predictions"])
predictions["Predictions"].sort_values(ascending = False)

Unnamed: 0
1891: Star Wars: Episode V - The Empire Strikes Back (1980)    4.760291
155: The Dark Knight (2008)                                    4.551454
122: The Lord of the Rings: The Return of the King (2003)      4.507637
77: Memento (2000)                                             4.472487
121: The Lord of the Rings: The Two Towers (2002)              4.400194
                                                                 ...   
3049: Ace Ventura: Pet Detective (1994)                        1.814717
8467: Dumb & Dumber (1994)                                     1.813320
9331: Clear and Present Danger (1994)                          1.801453
672: Harry Potter and the Chamber of Secrets (2002)            1.801453
4327: Charlie's Angels (2000)                                  1.439808
Name: Predictions, Length: 100, dtype: float64